In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# iPLS

This notebook will implement interval partial least squares.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import smelly_rats

# Loading the data

In [ ]:
xl = pd.ExcelFile('../data/onions.xls')

In [ ]:
hnmr_spectra = (
    pd.read_excel(xl, sheet_name=0, index_col=0)
    .rename(columns=lambda col: col.replace("'", ""))
)
print('HNMR spectra shape:', hnmr_spectra.shape)
hnmr_spectra.head()

In [ ]:
target = pd.read_excel(xl, sheet_name=1, usecols=range(10, 15))
print(target.shape)
target.head()

# Interval partial leas squares

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from smelly_rats.preproccessing import ParetoTransformer, ColumnSliceSelector

In [ ]:
def misclassification(y, y_hat):
    y = y.values if hasattr(y, 'values') else y
    return (y.flatten() != y_hat.flatten().round().astype(int)).sum()
misclassification_scorer = make_scorer(misclassification)


def rmse(y, y_hat):
    y = y.values if hasattr(y, 'values') else y
    return np.sqrt(np.square(y.flatten() - y_hat.flatten()).mean())
rmse_scorer = make_scorer(rmse)

In [ ]:
num_intervals = 20      # number of spectral intervals

mod = Pipeline([
    ('scaling', ParetoTransformer()),
    ('spectral_band', ColumnSliceSelector(10, 0)),
    ('regressor', PLSRegression(n_components=7, scale=False))
])

intervals = np.linspace(
    hnmr_spectra.index.max(), 
    hnmr_spectra.index.min(), 
    num_intervals + 1
)

param_grid = {
    'spectral_band': (
        [ColumnSliceSelector(start, stop) for start, stop in zip(intervals[:-1], intervals[1:])] 
        + [ColumnSliceSelector(intervals.max(), intervals.min())]
    )
}

grid = GridSearchCV(
    mod,
    param_grid,
    cv=5,
    iid=False,
    scoring=rmse_scorer,
    return_train_score=True
)

grid.fit(hnmr_spectra.T, target['y'])

In [ ]:
fig, axes = plt.subplots(2, figsize=(16, 16))

cv_results = (
    pd.DataFrame(grid.cv_results_)
    .assign(start=lambda df: [row['spectral_band'].start for row in df['params']],
            stop=lambda df: [row['spectral_band'].stop for row in df['params']])
)

for ax, train_test in zip(axes, ('train', 'test')):
    global_score = (
        cv_results
        .loc[lambda df: df['start'] == df['start'].max()]
        .loc[lambda df: df['stop'] == df['stop'].min()]
        [f'mean_{train_test}_score']
        .values[0]
    )
    
    for _, row in cv_results.iterrows():
        start = row['start']
        stop = row['stop']
        height = row[f'mean_{train_test}_score']
        yerr = row[f'std_{train_test}_score']

        if start == intervals.max() and stop == intervals.min():        # global
            ax.axhline(height, linestyle='--', color='black', alpha=0.8)
            ax.axhline(height - yerr, linestyle='-.', color='black', alpha=0.8)
            ax.axhline(height + yerr, linestyle='-.', color='black', alpha=0.8)
        else:                                                          # interval
            alpha = 0.3 if height > global_score else 0.8
            ax.bar(
                stop, 
                height=height, 
                width=(start - stop), 
                align='edge', 
                color='grey',
                alpha=alpha,
                edgecolor='black',
                yerr=yerr,
                capsize=7,
            )

    # Also plot spectra for clarity
    plotr = (
        hnmr_spectra
        .mean(axis=1)
        .transform(lambda series: series / series.max())
        .transform(lambda series: series * cv_results[f'mean_{train_test}_score'].max())
    )
    plotr.plot(ax=ax, color='black')

    ax.set_xlim([intervals.max(), intervals.min()])
    if ax == axes[-1]:
        ax.set_xlabel('ppm')
    else:
        ax.set_xlabel('')
    ax.set_ylabel('RMSECV')
    ax.set_xticks(np.arange(np.ceil(intervals.min()), np.ceil(intervals.max())))
    ax.set_title(f'{train_test} data')
    
fig.suptitle(f'Interval PLS on the H NMR spectra with {num_intervals} spectral bands.')
# fig.tight_layout();